# Instalação e importação de dependências

In [ ]:
# Framework RAG
!pip install langchain langchain-community langchain-huggingface

# Embeddings e Modelos (Hugging Face)
!pip install sentence-transformers transformers torch accelerate datasets

# Banco de dados vetorial
!pip install chromadb faiss-cpu

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import torch
from datasets import load_dataset
import numpy as np

# Leitura do dataset utilizado como RAG

## Visualização do dataset

O dataset escolhido foi o **SQuAD v2** (Stanford Question Answering Dataset 2.0), que é uma versão aprimorada do famoso SQuAD 1.1, amplamente usado para treinamento e avaliação de modelos de compreensão de linguagem natural (NLP), especialmente em tarefas de Pergunta e Resposta (Question Answering).

**Formato dos dados:** cada amostra contém:

* **context:** o texto base (geralmente um parágrafo da Wikipédia);

* **question:** a pergunta feita sobre o contexto;

* **answers:** contém as respostas corretas (text e answer_start);

* **id:** identificador único da amostra.

In [ ]:
dataset = load_dataset("squad_v2", split="train[:100]") # Primeiras 100 entradas

# Transformando em dataframe do pandas
df = dataset.to_pandas()
print(df.shape)
df

## Preparação dos dados

In [ ]:
def load_sample_data():
  """Carrega um dataset de QA para demonstração"""
  dataset = load_dataset("squad_v2", split="train[:100]") # Primeiras 100 entradas
  documents = []

  for item in dataset:
    # Criar documentos a partir do contexto
    doc = Document(
      page_content=item["context"],
      metadata={
        "title": f"Artigo_{item['id']}",
        "question": item["question"],
        "answers": item["answers"]
      }
    )
    documents.append(doc)

  return documents

In [ ]:
# Carregando os dados
documents = load_sample_data()
print(f"Carregados {len(documents)} documentos")

for i, doc in enumerate(documents[:5]):
  print(f"Documento {i+1}: {doc.page_content[:100]}...")

# Prepearação do banco de dados vetorial

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def setup_retrieval_system(documents):
    """Configura o sistema de recuperação vetorial.
    Transforma documentos de texto em vetores numéricos (embeddings) e os armazena em um banco vetorial (FAISS), permitindo buscas semânticas eficientes."""

    # Divide documentos em chunks menores (até 300 caracteres com 50 caracteres de sobreposição entre os trechos consecutivos)
    text_splitter = CharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separator="\n"
    )

    texts = text_splitter.split_documents(documents)
    print(f"Documentos divididos em {len(texts)} chunks")

    # Carrega modelo de embeddings do Hugging Face (compatível com LangChain)
    # all-MiniLM-L6_v2: modelo pequeno e rápido do Hugging Face para similaridade semântica e tarefas de recuperação
    embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

    # Cria índice de busca vetorial com FAISS
    vector_store = FAISS.from_documents(texts, embedding_model)

    return vector_store, embedding_model, texts

# Inicializa o sistema de recuperação
vector_store, embedding_model, text_chunks = setup_retrieval_system(documents)

# Definindo sistema RAG

In [ ]:
# Configurando o modelo de QA do Hugging Face
def setup_generator():
  """Configura o pipeline de Question Answering"""

  # Definindo um pipeline pré-pronto
  # distilbert-base-cased-distilled-squad: modelo treinado especificamente no dataset escolhido
  qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    tokenizer="distilbert-base-cased-distilled-squad"
  )

  return qa_pipeline

# Inicializando o modelo de QA
print("Carregando modelo de Question Answering...")
qa_pipeline = setup_generator()
print("Modelo carregado com sucesso!")

In [ ]:
# Implementação o sistema RAG completo
class SimpleRAGSystem:
  def __init__(self, vector_store, qa_pipeline, top_k=4):
    self.vector_store = vector_store
    self.qa_pipeline = qa_pipeline
    self.top_k = top_k

  def retrieve_documents(self, question):
    """Recupera os documentos mais relevantes"""
    # Buscar documentos similares
    docs = self.vector_store.similarity_search(question, k=self.top_k)
    return docs

  def create_context(self, documents):
    """Combina os documentos em um único contexto"""
    contexts = [doc.page_content for doc in documents]
    return "\n\n".join(contexts)

  def generate_answer(self, question, context):
    """Gera resposta usando o pipeline de QA"""
    try:
      # Formata entrada para o pipeline
      qa_input = {
      'question': question,
      'context': context
      }

      # Gera resposta
      result = self.qa_pipeline(qa_input)
      return result
    except Exception as e:
      return {"answer": f"Erro ao gerar resposta: {str(e)}", "score": 0.0}

  def query(self, question):
    """Método principal para fazer consultas ao sistema RAG"""
    print(f"Pergunta: {question}")

    # Fase de recuperação
    relevant_docs = self.retrieve_documents(question)
    print(f"Documentos recuperados: {len(relevant_docs)}")

    # Fase de criação de contexto
    context = self.create_context(relevant_docs)

    # Fase de geração da resposta
    answer_result = self.generate_answer(question, context)
    return {
    "question": question,
    "answer": answer_result["answer"],
    "confidence": answer_result["score"],
    "source_documents": relevant_docs,
    "context_used": context
    }

# Inicializando o sistema RAG
rag_system = SimpleRAGSystem(vector_store, qa_pipeline, top_k=5) # Defina top_k como o número de documentos utilizados como contexto para a resposta

# Testes

In [ ]:
# Testando o sistema com exemplos práticos
def test_rag_system():
  """Testa o sistema RAG com várias perguntas"""

  test_questions = [
  "How many years did Beyoncé's career last?",
  "How much records did Beyoncé sold as a solo artist?",
  "What are the themes of Beyoncé songs?",
  "Where was Beyoncé born?",
  "Who is Beyoncé?"
  ]

  for question in test_questions:
    print("\n" + "="*60)
    result = rag_system.query(question)

    print(f"Resposta: {result['answer']}")
    print(f"Confiança: {result['confidence']:.4f}")
    print(f"Fontes usadas:")

    for i, doc in enumerate(result['source_documents']):
      print(f" {i+1}. {doc.page_content}...")
      print("-"*60)

# Executar testes
test_rag_system()

# Avaliação do sistema

In [ ]:
# Avaliação do sistema
def evaluate_rag_system():
  """Avalia o desempenho do sistema RAG"""

  # Perguntas de teste com respostas esperadas
  test_cases = [
  {
  "question": "How many years did Beyoncé's career last?",
  "expected_keywords": ["years", "span", "Beyoncé", "19"]
  },
  {
  "question": "What are the themes of Beyoncé songs?",
  "expected_keywords": ["themes", "songs", "Beyoncé", "darker"]
  }
  ]

  print("Avaliando Sistema RAG")
  print("-" * 40)

  for i, test_case in enumerate(test_cases):
    result = rag_system.query(test_case["question"])

    print(f"\nTeste {i+1}:")
    print(f"Pergunta: {test_case['question']}")
    print(f"Resposta: {result['answer']}")
    print(f"Confiança: {result['confidence']:.4f}")

    # Verificar se contém palavras-chave esperadas
    answer_lower = result['answer'].lower()
    keywords_found = [kw for kw in test_case["expected_keywords"] if kw
    in answer_lower]

    print(f"Palavras-chave encontradas: {keywords_found}")
    print(f"Score:{len(keywords_found)}/{len(test_case['expected_keywords'])}\n")

# Executar avaliação
evaluate_rag_system()

# Conclusão

O modelo é capaz de utilizar a base vetorial como contexto, caracterizando-se como um modelo RAG funcional. No entanto, as respostas nem sempre são corretas e claras, embora o modelo utilizado tenha sido treinado com a base de dados selecionada. Melhorias na base e no modelo, bem como a utilização de engenharia de prompts poderiam melhorar o desempenho do modelo, o que será feito em um trabalho futuro.